In [21]:
!pip install python-coinmarketcap

from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import string
import json
import re 
import datetime
import os
import codecs
import nltk
from pathlib import Path
import shutil
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import coinmarketcapapi

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from string import punctuation
from datetime import date, timedelta
nltk.download('stopwords')


import glob
import plotly 
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objs as go

import requests
from bs4 import BeautifulSoup
from google.colab import drive


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
consumer_key = "jDE9q36HhxaT9pZWdEcNPYM6o"
consumer_secret = "PHSoZwsNu7vzs1fv1iBjbeb5bMJuk6pO8MhJBclstAQV5PAht6"
access_token  = "2339839940-eHCIFZpXd9yShBE7MkHeuZQrjYZPibvpmaF6sR1"
access_token_secret = "GDoVMQdCpenEGZhXqa0VhdapEM5C3ULihMXGyUQ1mTlZ3"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True)
type(api)

# All search terms we can use
search_term_xrp = '#$XRP OR #XRP OR #xrp OR #Ripple'
search_term_btc = '#BTC OR #Btc OR #btc OR #bitcoin'
search_term_eth = '#ETH OR #Eth OR #eth OR #ethereum'
search_term_bnb = '#BNB OR #Binance OR #bnb OR #binance'
search_term_doge = '#DOGE OR #Doge OR #doge'
search_term_litecoin = '#LTC OR #litecoin OR #ltc'
search_term_bitcoincash = '#BitcoinCash OR #BCH OR #bitcoincash'

no_of_tweets = 20

# setting specified timeframe to pull tweets from 
start_date = '2022-12-05T02:11:02'
end_date = '2022-12-06T03:11:01'
tweets = tweepy.Cursor(api.search, q = search_term_xrp, lang = "en", since = start_date, until = end_date).items(no_of_tweets)
print(tweets)

In [23]:
xrp_tweets = 'ripple_tweets.csv'
btc_tweets = "btc_tweets.csv"
eth_tweets = "eth_tweets.csv"
bnb_tweets = "bnb_tweets.csv"
doge_tweets = "doge_tweets.csv"
litecoin_tweets = "litecoin_tweets.csv"
bitcoincash_tweets = "bitcoincash_tweets.csv"


# only including column for original text and created at
COLS = ['created_at','original_text']

In [24]:
# Function to write tweets to separate csv files depending on the search terms of each crypto currency
def write_tweets(search_term, file):
  df = pd.DataFrame(columns = COLS)
  tweets = tweepy.Cursor(api.search, q = search_term, lang = "en", since = start_date, until = end_date).items(no_of_tweets)
  csvFile = open(file, 'a')
  csvWriter = csv.writer(csvFile)
  df.to_csv(csvFile, mode = 'a', columns = COLS, encoding = "utf-8", index  = False)
  for tweet in tweets:
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    print(tweet.created_at, tweet.text)


In [25]:
# conduct same level of preprocessing as done in logestic regression model remove capitalization, stopwords, punctuation, and twitter link
def process(text): 
  text = str(text)
  text = text.lower()
  text = re.sub(r'@[^\s]+', '', text) # remove username
  text = re.sub(r'http\S+', '', text, flags=re.MULTILINE) # remove http link
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  procList = [word for word in nopunc.split() if word not in stopwords.words('english')]
  return ' '.join(procList)

# Change parameters of search term and file type to whatever you intend to query for (commented out all other 4 currencies for testing purposes)

write_tweets(search_term_btc, btc_tweets)
write_tweets(search_term_eth, eth_tweets)
write_tweets(search_term_xrp, xrp_tweets)
write_tweets(search_term_bnb, bnb_tweets)
write_tweets(search_term_doge, doge_tweets)
write_tweets(search_term_litecoin, litecoin_tweets)
write_tweets(search_term_bitcoincash, bitcoincash_tweets)

2022-12-05 23:59:59 RT @QuaiNetwork: We asked Open AI's #ChatGPT what it thought of the unique architecture that #Quai has implemented to solve the blockchain…
2022-12-05 23:59:59 RT @bitindichain: Bitindi Chain $BNI Validator / Delegator Platform🎊On DEC 9th 2022
Buy Now: https://t.co/TPKD1cwr3j
#Blockchain #Cryptocur…
2022-12-05 23:59:58 $GTO  huge new coming, will pump more then #Bitcoin
2022-12-05 23:59:58 RT @TheCryptoLark: The video banks don't want you to see

#banking #bitcoin #crypto #cryptocurrency #btc #bankingsystem #fractionalreserve…
2022-12-05 23:59:57 They accomplish the opposite of what they set out to because their intentions are against your God-given rights. So… https://t.co/VPGsByZbdF
2022-12-05 23:59:57 I believe that #Bitcoin will succeed to its fullest extent, and that the best way to help yourself and everyone els… https://t.co/bMsttDhti1
2022-12-05 23:59:56 Your situation is not as serious as you are dealing with

 Inbox me if you still want your account!

 I h

In [26]:
# read in data frame to view original text compared to pre-processed text
data = pd.read_csv(btc_tweets, low_memory= False)
data['text_clean'] = data['original_text'].apply(lambda row: process(row))
dataBTC = data[['original_text', 'text_clean']]
dataBTC

# to view the dataframe for any other currencty simply change the first parameter in pd_read_csv to the tweets you'd wish to view as shown below
# dataXRP = pd.read_csv(xrp_tweets, low_memory = False)
# dataETH = pd.read_csv(eth_tweets, low_memory = False)
# dataBNB = pd.read_csv(bnb_tweets, low_memory = False)
# dataDOGE = pd.read_csv(doge_tweets, low_memory = False)
# dataBCH = pd.read_csv(bitcoincash_tweets, low_memory = False)
# dataLTC = pd.read_csv(litecoin_tweets, low_memory = False)

,original_text,text_clean
0,"b""RT @QuaiNetwork: We asked Open AI's #ChatGPT...",brt asked open ais chatgpt thought unique arch...
1,b'RT @bitindichain: Bitindi Chain $BNI Validat...,brt bitindi chain bni validator delegator plat...
2,"b'$GTO huge new coming, will pump more then #...",bgto huge new coming pump bitcoin
3,"b""RT @TheCryptoLark: The video banks don't wan...",brt video banks dont want seennbanking bitcoin...
4,b'They accomplish the opposite of what they se...,bthey accomplish opposite set intentions godgi...
5,b'I believe that #Bitcoin will succeed to its ...,bi believe bitcoin succeed fullest extent best...
6,b'Your situation is not as serious as you are ...,byour situation serious dealing withnn inbox s...
7,b'RT @jihanicorn: $150 | 2.250.000 IDR \xe2\x8...,brt 150 2250000 idr xe2x80xa2 24 hours xf0x9fx...
8,b'RT @TheCryptoLark: What if I told you...\n\n...,brt told younnxf0x9fx94xa5 0 spot trading fees...
9,b'RT @jihanicorn: $150 | 2.250.000 IDR \xe2\x8...,brt 150 2250000 idr xe2x80xa2 24 hours xf0x9fx...


In [27]:
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [28]:
!ls /content/drive/MyDrive/CS410Project  

coin_interface.ipynb  tfidf_transformer.pkl
logreg.pkl	      train_data.csv
logreg_train.ipynb    twitter_crypto_sentiment_main.ipynb


In [29]:
# Load trained logreg model state
with open('/content/drive/MyDrive/CS410Project/logreg.pkl', 'rb') as f:
    logreg = pickle.load(f)
    
# with open('/content/drive/MyDrive/CS410Project/tfidf_transformer.pkl', 'rb') as f: 
#    tfidf_transformer = pickle.load(f)
# The above approach is running into a dependency issue with scipy due to the usage of a sparse matrix data struct. 
# Installing the library in the notebook doesn't seem to work so we recreate the transformer from the train data


# Load train dataset to recreate the tf-idf transformer
data['text'] = data['text_clean'] # data can be changed 2 cells above to ping any specific crypto currency where tweets are pulled for 
train_data = pd.read_csv('/content/drive/MyDrive/CS410Project/train_data.csv', low_memory=False)
max_vocab = 4000 # don't change
tfidf_transformer = TfidfVectorizer(max_features=max_vocab, ngram_range=(1,2)).fit(train_data['text'].values.astype('U'))

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator LogisticRegression from version 1.1.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [30]:
x = tfidf_transformer.transform(data['text'].values.astype('U'))
y_hat = y_hat_train = logreg.predict(x)


In [31]:
print(data['text'])

0     brt asked open ais chatgpt thought unique arch...
1     brt bitindi chain bni validator delegator plat...
2                     bgto huge new coming pump bitcoin
3     brt video banks dont want seennbanking bitcoin...
4     bthey accomplish opposite set intentions godgi...
5     bi believe bitcoin succeed fullest extent best...
6     byour situation serious dealing withnn inbox s...
7     brt 150 2250000 idr xe2x80xa2 24 hours xf0x9fx...
8     brt told younnxf0x9fx94xa5 0 spot trading fees...
9     brt 150 2250000 idr xe2x80xa2 24 hours xf0x9fx...
10    brt stoked see team following conference lives...
11    bethereum price update nneth 125998 usdnbitcoi...
12    bxf0x9fxa7xb5nnthe government doesnxe2x80x99t ...
13    brt 24 hour 1000 bitcoin giveawaynnto entern l...
14    brt xf0x9fx8ex81x2 500 giveaway mexc xf0x9fx8e...
15    brt 150 2250000 idr xe2x80xa2 24 hours xf0x9fx...
16    btell restore trust wallet phrase restore coin...
17    brt 150 2250000 idr xe2x80xa2 24 hours xf0

In [32]:
y_hat

array([ 1.,  0.,  0.,  0.,  1.,  1.,  1., -1., -1., -1.,  0., -1.,  1.,
        0.,  0., -1.,  0., -1.,  0.,  0.])

### Data Visualization

In [33]:
def percentage(part, whole):
    return 100 * float(part)/float(whole)

### Read Tweets from CSV file and run prediction on them using the trained model.

In [34]:
# initializing dataframes
crypto_name = "bitcoin" # we can change this to have user input value instead 
df_noOfTweets = pd.DataFrame(columns=['date','positive', 'neutral', 'negative', 'total']) # for the barchart of no. of tweets
df_polarity_line = pd.DataFrame(columns=['date','polarity']) # for the polarity line chart
# Tweets about each crypto is taken from saved txt files with dates ranging from 2017-09-01 to 2017-11-30
# Tweets are collected by selenium automated script written in python
for f in glob.glob( '/content/drive/MyDrive/database/Tweets/'+ crypto_name +'/'+ crypto_name + '*.txt'):
  df_datawise = pd.read_csv(f, sep='\t', encoding='utf-8', parse_dates=True)
  polarity = 0.00
    # df_datawise = pd.read_csv(f, sep='\t', encoding='utf-8', parse_dates=True)
    # df_datawise = pd.read_csv(btc_tweets, low_memory= False)
  df_datawise['Tweets'] = df_datawise['Tweets'].apply(lambda row: process(row))
  x = tfidf_transformer.transform(df_datawise['Tweets'].values.astype('U'))
  y_hat =  logreg.predict(x)
  polarity = 0.00
  neutral = positive = negative = 0
  for idx,row in df_datawise.iterrows():
      tweet_polarity = y_hat[idx]
      polarity += y_hat[idx]  # adding up polarities to find the average later
      
      # adding reaction of how people are reacting to find average later
      if (tweet_polarity == 1): 
          #print(positive) 
          positive += 1
          #raise NotImplementedError
      else:
          #print(negative)
          negative += 1
          #raise NotImplementedError
  positive = percentage(positive, len(df_datawise.index))
  negative = percentage(negative, len(df_datawise.index))
  neutral = percentage(neutral, len(df_datawise.index))

  # below df format is for barchart with sentiments
  # df_temp_df_noOfTweets = pd.DataFrame({'date': df_datawise.iloc[0]['date'], 'positive': ((positive/100)*len(df_datawise.index)),'neutral': ((neutral/100)*len(df_datawise.index)), 'negative': ((negative/100)*len(df_datawise.index)), 'total': len(df_datawise.index)}, index=[0])
  # df_noOfTweets = df_barchart.append(df_temp_df_noOfTweets, sort=True, ignore_index=True)

  # below df format is for polarity_line with sentiments
  df_temp_polarity_line = pd.DataFrame({'date': df_datawise.iloc[0]['date'], 'polarity': polarity}, index=[0])
  df_polarity_line = df_polarity_line.append(df_temp_polarity_line, sort=True, ignore_index=True)
print(df_polarity_line)
    

          date  polarity
0   2017-09-16      37.0
1   2017-10-06     143.0
2   2017-11-09     250.0
3   2017-10-07     124.0
4   2017-11-22     360.0
..         ...       ...
86  2017-10-29     159.0
87  2017-09-21     175.0
88  2017-09-01     142.0
89  2017-10-18     179.0
90  2017-11-19     293.0

[91 rows x 2 columns]


In [35]:
table = ff.create_table(df_noOfTweets)
# plotly.offline.iplot(table, filename='df_noOfTweets')
# =================== #
# no_of_tweets TABLE  #
# =================== #

### Coinmarketcap Package installation

### Coin Market APIs to  get Closing Price

In [36]:
cmc = coinmarketcapapi.CoinMarketCapAPI('b18b870d-d87e-4130-8243-6220c170e475')
data_id_map = cmc.cryptocurrency_map()
#print(data_id_map.data)
pdf = pd.DataFrame(data_id_map.data, columns =['id','name','symbol'])
#pdf.set_index('symbol',inplace=True)
id = pdf[pdf['symbol'] == 'BTC'].id
print(pdf)

         id         name   symbol
0         1      Bitcoin      BTC
1         2     Litecoin      LTC
2         3     Namecoin      NMC
3         4    Terracoin      TRC
4         5     Peercoin      PPC
...     ...          ...      ...
8957  22913   Dives Defi      DDF
8958  22919       All in    ALLIN
8959  22921  Pi Protocol      PIP
8960  22922      Healium      HLM
8961  22923   PlanetCats  CATCOIN

[8962 rows x 3 columns]


In [37]:
# ===================================== #
# Getting the closing price and volume  #
# ===================================== #
#url='https://coinmarketcap.com/currencies/'+ crypto_name +'/historical-data/?start=20170901&end=20171130'
#url="https://coinmarketcap.com/currencies/"+ crypto_name +"/historical-data/?start=20170901&end=20171130"

api_url= 'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/historical?id=1&convertId=2781&timeStart=1504249200&timeEnd=1512162061'
r = requests.get(api_url)
data = []
for item in r.json()['data']['quotes']:
    close = item['quote']['close']
    volume =item['quote']['volume']
    date=item['quote']['timestamp']
    data.append([close,volume,date])

cols = ["close", "volume","date"]

df = pd.DataFrame(data, columns= cols) 
df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d"))
df_marketcap_sentiment_line = pd.merge(df_polarity_line, df, left_on='date', right_on='date')
# ======================================= #
# Polarity, Vol. and Closing Price TABLE  #
# ======================================= #
table = ff.create_table(df_marketcap_sentiment_line)
#plotly.offline.iplot(table, filename='df_marketcap_sentiment_line')
#print(df_marketcap_sentiment_line.sort_values(by=["date"]))

In [38]:
# ==================================================================== #
# Bar Graphs of Each Sentiment, Volume, and Closing Average over Time  #
# ==================================================================== #

# df_market_cap_sentiment_line name of the df we're working with that has all the information 
#  columns go as such: close, volume, date

# barchart1 = make_subplots(specs =[[{"secondary_y": True}]])
barchart1 = px.bar(
    data_frame = df_marketcap_sentiment_line,
    title = "Crypto Closing Averages over Time",
    x = "date",
    y = "close",
    opacity = 0.9, 
    orientation = "v",
    barmode = 'relative',
    text = 'close',
    color = 'close',
)

barchart2 = px.bar(
    data_frame = df_marketcap_sentiment_line,
    title = "Sentiment Polarity over Time",
    x = "date",
    y = "polarity",
    opacity = 0.9, 
    orientation = "v",
    barmode = 'relative',
    text = 'close',
    color = 'polarity',
)
pio.show(barchart2)

barchart3 = px.bar(
    data_frame = df_marketcap_sentiment_line,
    title = "Volume over Time",
    x = "date",
    y = "volume",
    opacity = 0.9,
    orientation = "v",
    barmode = 'relative',
    text = 'close',
    color = 'volume',
)
pio.show(barchart3)

# data1 = [barchart1, barchart2]
pio.show(barchart1)



### Plot for Polarity Vs Closing Price

In [39]:
# ============================================== #
# Plot Polarity, Price in USD(Close) and Volume  #
# ============================================== #

trace1 = go.Scatter(
    x=df_marketcap_sentiment_line['date'],
    y=df_marketcap_sentiment_line['close'],
    mode = 'markers',
    name='Closing Price'
)
trace2 = go.Scatter(
    x=df_marketcap_sentiment_line['date'],
    y=df_marketcap_sentiment_line['polarity'],
    mode = 'markers',
    name='Polarity',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Sentiment Polarity Vs. Closing Price (in USD)',
    yaxis=dict(
        title='Closing Price.'
    ),
    yaxis2=dict(
        title='Polarity',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)

# fig.show()

plotly.offline.iplot(fig, filename='Sentiment_vs_Market.html')